# Introduction

In [1]:
import os
from matplotlib import pyplot
import pandas
from pathlib import Path
import sys
from tqdm import tqdm


Next we import dependencies we need to use.

In [2]:
EC = str(Path("~/proj/encoded_client").expanduser())
if EC not in sys.path:
    sys.path.append(EC)
    
from encoded_client.encoded import ENCODED

server = ENCODED("www.encodeproject.org")

Pandas can directly read urls, and the encode publication set has a file with a list of all the files and what experiments they're from. Unfortunately it lacks much of the biosample level metadata. But at least given the File dataset, we can go retrieve more information.

In [3]:
file_metadata = pandas.read_csv("https://www.encodeproject.org/documents/ab75e52f-64d9-4c39-aea0-15372479049d/@@download/attachment/ENCSR574CRQ_metadata.tsv", sep="\t")
file_metadata


,File accession,File dataset,File type,File format,File output type,Assay term name,Biosample term id,Biosample term name,Biosample type,File target,...,Project,Lab,md5sum,dbxrefs,File download URL,Assembly,File status,Derived from,S3 URL,Size
0,ENCFF920CNZ,ENCSR304RDL,fastq,fastq,reads,RNA-seq,UBERON:0001890,forebrain,tissue,NaN,...,ENCODE,"Barbara Wold, Caltech",23d4d1fcdbfe3e477896f77e5208b3c0,SRA:SRR4421602,/files/ENCFF920CNZ/@@download/ENCFF920CNZ.fast...,NaN,released,NaN,https://encode-public.s3.amazonaws.com/2016/07...,2221837463
1,ENCFF320FJX,ENCSR304RDL,fastq,fastq,reads,RNA-seq,UBERON:0001890,forebrain,tissue,NaN,...,ENCODE,"Barbara Wold, Caltech",7d3a3b102641187fee176b020d27223c,SRA:SRR4421603,/files/ENCFF320FJX/@@download/ENCFF320FJX.fast...,NaN,released,NaN,https://encode-public.s3.amazonaws.com/2016/07...,2193429461
2,ENCFF528EVC,ENCSR304RDL,fastq,fastq,reads,RNA-seq,UBERON:0001890,forebrain,tissue,NaN,...,ENCODE,"Barbara Wold, Caltech",edf87120238bbfbf117482ae6172c4a4,SRA:SRR4421600,/files/ENCFF528EVC/@@download/ENCFF528EVC.fast...,NaN,released,NaN,https://encode-public.s3.amazonaws.com/2016/07...,1969887894
3,ENCFF663SNC,ENCSR304RDL,fastq,fastq,reads,RNA-seq,UBERON:0001890,forebrain,tissue,NaN,...,ENCODE,"Barbara Wold, Caltech",882e38113300a00cc2e7e3f0fdcf24a8,SRA:SRR4421601,/files/ENCFF663SNC/@@download/ENCFF663SNC.fast...,NaN,released,NaN,https://encode-public.s3.amazonaws.com/2016/07...,1989101437
4,ENCFF133MSL,ENCSR304RDL,bam,bam,alignments,RNA-seq,UBERON:0001890,forebrain,tissue,NaN,...,ENCODE,ENCODE Processing Pipeline,bfda8d47bf8ac1e72746e14a7765d597,NaN,/files/ENCFF133MSL/@@download/ENCFF133MSL.bam,mm10,released,"ENCFF320FJX, ENCFF920CNZ, ENCFF533JRE",https://encode-public.s3.amazonaws.com/2016/08...,5174602843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,ENCFF584AIQ,ENCSR739PEB,bigWig,bigWig,signal of all reads,RNA-seq,UBERON:0002369,adrenal gland,tissue,NaN,...,ENCODE,ENCODE Processing Pipeline,bcb98dcbc0caa48642592b9f91705bde,NaN,/files/ENCFF584AIQ/@@download/ENCFF584AIQ.bigWig,mm10,released,ENCFF752FZT,https://encode-public.s3.amazonaws.com/2016/08...,187310904
1195,ENCFF465YOS,ENCSR739PEB,tsv,tsv,gene quantifications,RNA-seq,UBERON:0002369,adrenal gland,tissue,NaN,...,ENCODE,ENCODE Processing Pipeline,9277ac793ae1285e953f409dfd1f87d4,NaN,/files/ENCFF465YOS/@@download/ENCFF465YOS.tsv,mm10,released,"ENCFF717NFD, ENCFF077IZN",https://encode-public.s3.amazonaws.com/2016/08...,8644586
1196,ENCFF396NRQ,ENCSR739PEB,tsv,tsv,transcript quantifications,RNA-seq,UBERON:0002369,adrenal gland,tissue,NaN,...,ENCODE,ENCODE Processing Pipeline,a9c42437793f4ad5221ddc2d0f2e93d8,NaN,/files/ENCFF396NRQ/@@download/ENCFF396NRQ.tsv,mm10,released,"ENCFF717NFD, ENCFF077IZN",https://encode-public.s3.amazonaws.com/2016/08...,16414585
1197,ENCFF415XCZ,ENCSR739PEB,tsv,tsv,transcript quantifications,RNA-seq,UBERON:0002369,adrenal gland,tissue,NaN,...,ENCODE,ENCODE Processing Pipeline,5a89b79b33780ee293ad00a676725a35,NaN,/files/ENCFF415XCZ/@@download/ENCFF415XCZ.tsv,mm10,released,"ENCFF717NFD, ENCFF522TMY",https://encode-public.s3.amazonaws.com/2016/08...,16393312


In [4]:
file_metadata["File download URL"] = file_metadata["File download URL"].apply(lambda x: server.prepare_url(x))

In [5]:
experiments = set(file_metadata["File dataset"])

# Web method

The encode portal does have a report view which returns a tsv file that you can customize with more metadata information, unfortunately I don't know of a direct way to go from a publication set, to the report.

So here we first construct a search url from all the experiment accession ids.

In [6]:
base = "https://www.encodeproject.org/search/?type=Experiment&accession="
url = base + "&accession=".join(experiments)

print(url)

https://www.encodeproject.org/search/?type=Experiment&accession=ENCSR943LKA&accession=ENCSR150CUE&accession=ENCSR667TOX&accession=ENCSR739PEB&accession=ENCSR343YLB&accession=ENCSR906YQZ&accession=ENCSR932TRU&accession=ENCSR648YEP&accession=ENCSR764OPZ&accession=ENCSR597UZW&accession=ENCSR559TRB&accession=ENCSR537GNQ&accession=ENCSR448MXQ&accession=ENCSR557RMA&accession=ENCSR420QTO&accession=ENCSR526SEX&accession=ENCSR466KZY&accession=ENCSR848GST&accession=ENCSR438XCG&accession=ENCSR178GUS&accession=ENCSR848HOX&accession=ENCSR719NAJ&accession=ENCSR538WYL&accession=ENCSR809VYL&accession=ENCSR970EWM&accession=ENCSR285WZV&accession=ENCSR823VEE&accession=ENCSR457RRW&accession=ENCSR216NEG&accession=ENCSR968QHO&accession=ENCSR160IIN&accession=ENCSR921PRX&accession=ENCSR049UJU&accession=ENCSR331XCE&accession=ENCSR017JEG&accession=ENCSR508GWZ&accession=ENCSR290RRR&accession=ENCSR020DGG&accession=ENCSR185LWM&accession=ENCSR096STK&accession=ENCSR307BCA&accession=ENCSR982MRY&accession=ENCSR504GEG&

From there on the Experiment search page you can click on "Report", then on "Columns" to select the columns of interest if you'd like to change them, and then finally on download TSV to retrieve a table with more of the experiment metadata.

# Programatic method

encoded_client is my utility that provides some wrappers to access the encode json api. There's also a package called encode_utils which is similar.

The actual request is https://www.encodeproject.org/experiments/ENCSR727FHP/?format=json (iterating over all the different experiment accession ids from the file metadata.

In [7]:
server = ENCODED("www.encodeproject.org")

This chunk of code requests the experiment json objects, which are pretty detailed and include information about the experiment, and the replicate objects that link to the library and biosample objects that make up the experiment.

The different objects hold different kinds of information, and in this case since we're looking for 

In [8]:
experiment_metadata = []
for accession in tqdm(experiments):
    experiment = server.get_json(accession)
    for replicate in experiment["replicates"]:
        library = replicate["library"]
        biosample = library["biosample"]
        biosample_ontology = biosample["biosample_ontology"]
        experiment_metadata.append({
            "experiment": experiment["accession"], 
            "biosample": biosample["accession"], 
            "biosample_term_name": biosample_ontology["term_name"], 
            "mouse_life_stage": biosample["mouse_life_stage"], 
            "age": biosample['model_organism_age'], 
            "age_units": biosample['model_organism_age_units'],
            "timepoint": "{} {} {}".format(biosample["mouse_life_stage"], biosample['model_organism_age'], biosample['model_organism_age_units'],)
        })
        # we just want the age & tissue inform which is the same for all the replicates
        break

experiment_metadata = pandas.DataFrame(experiment_metadata)

100%|████████████████████████████████████████████████████████████████████████████████| 78/78 [00:23<00:00,  3.27it/s]


In [9]:
experiment_metadata

,experiment,biosample,biosample_term_name,mouse_life_stage,age,age_units,timepoint
0,ENCSR943LKA,ENCBS626UIA,hindbrain,embryonic,10.5,day,embryonic 10.5 day
1,ENCSR150CUE,ENCBS452TDN,heart,embryonic,12.5,day,embryonic 12.5 day
2,ENCSR667TOX,ENCBS310ESD,neural tube,postnatal,0,day,postnatal 0 day
3,ENCSR739PEB,ENCBS386FVC,adrenal gland,postnatal,0,day,postnatal 0 day
4,ENCSR343YLB,ENCBS825LGT,midbrain,embryonic,14.5,day,embryonic 14.5 day
...,...,...,...,...,...,...,...
73,ENCSR928OXI,ENCBS883PNF,neural tube,embryonic,14.5,day,embryonic 14.5 day
74,ENCSR062VTB,ENCBS899UOK,kidney,embryonic,15.5,day,embryonic 15.5 day
75,ENCSR611PTP,ENCBS966PAM,liver,embryonic,15.5,day,embryonic 15.5 day
76,ENCSR636CWO,ENCBS900XQR,embryonic facial prominence,embryonic,15.5,day,embryonic 15.5 day


In [10]:
experiment_metadata.to_csv("ENCSR574CRQ_biosample.tsv", sep="\t", index=None)

In [11]:
merged_metadata = file_metadata.merge(
    experiment_metadata.drop("biosample", axis=1), left_on="File dataset", right_on="experiment").drop(
    "experiment", axis=1).dropna(how="all", axis=1)
merged_metadata

,File accession,File dataset,File type,File format,File output type,Assay term name,Biosample term id,Biosample term name,Biosample type,Dataset accession,...,Assembly,File status,Derived from,S3 URL,Size,biosample_term_name,mouse_life_stage,age,age_units,timepoint
0,ENCFF920CNZ,ENCSR304RDL,fastq,fastq,reads,RNA-seq,UBERON:0001890,forebrain,tissue,ENCSR574CRQ,...,NaN,released,NaN,https://encode-public.s3.amazonaws.com/2016/07...,2221837463,forebrain,embryonic,10.5,day,embryonic 10.5 day
1,ENCFF320FJX,ENCSR304RDL,fastq,fastq,reads,RNA-seq,UBERON:0001890,forebrain,tissue,ENCSR574CRQ,...,NaN,released,NaN,https://encode-public.s3.amazonaws.com/2016/07...,2193429461,forebrain,embryonic,10.5,day,embryonic 10.5 day
2,ENCFF528EVC,ENCSR304RDL,fastq,fastq,reads,RNA-seq,UBERON:0001890,forebrain,tissue,ENCSR574CRQ,...,NaN,released,NaN,https://encode-public.s3.amazonaws.com/2016/07...,1969887894,forebrain,embryonic,10.5,day,embryonic 10.5 day
3,ENCFF663SNC,ENCSR304RDL,fastq,fastq,reads,RNA-seq,UBERON:0001890,forebrain,tissue,ENCSR574CRQ,...,NaN,released,NaN,https://encode-public.s3.amazonaws.com/2016/07...,1989101437,forebrain,embryonic,10.5,day,embryonic 10.5 day
4,ENCFF133MSL,ENCSR304RDL,bam,bam,alignments,RNA-seq,UBERON:0001890,forebrain,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF320FJX, ENCFF920CNZ, ENCFF533JRE",https://encode-public.s3.amazonaws.com/2016/08...,5174602843,forebrain,embryonic,10.5,day,embryonic 10.5 day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,ENCFF584AIQ,ENCSR739PEB,bigWig,bigWig,signal of all reads,RNA-seq,UBERON:0002369,adrenal gland,tissue,ENCSR574CRQ,...,mm10,released,ENCFF752FZT,https://encode-public.s3.amazonaws.com/2016/08...,187310904,adrenal gland,postnatal,0,day,postnatal 0 day
1195,ENCFF465YOS,ENCSR739PEB,tsv,tsv,gene quantifications,RNA-seq,UBERON:0002369,adrenal gland,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF077IZN",https://encode-public.s3.amazonaws.com/2016/08...,8644586,adrenal gland,postnatal,0,day,postnatal 0 day
1196,ENCFF396NRQ,ENCSR739PEB,tsv,tsv,transcript quantifications,RNA-seq,UBERON:0002369,adrenal gland,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF077IZN",https://encode-public.s3.amazonaws.com/2016/08...,16414585,adrenal gland,postnatal,0,day,postnatal 0 day
1197,ENCFF415XCZ,ENCSR739PEB,tsv,tsv,transcript quantifications,RNA-seq,UBERON:0002369,adrenal gland,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF522TMY",https://encode-public.s3.amazonaws.com/2016/08...,16393312,adrenal gland,postnatal,0,day,postnatal 0 day


In [12]:
merged_metadata.columns

Index(['File accession', 'File dataset', 'File type', 'File format',
       'File output type', 'Assay term name', 'Biosample term id',
       'Biosample term name', 'Biosample type', 'Dataset accession',
       'Dataset date released', 'Project', 'Lab', 'md5sum', 'dbxrefs',
       'File download URL', 'Assembly', 'File status', 'Derived from',
       'S3 URL', 'Size', 'biosample_term_name', 'mouse_life_stage', 'age',
       'age_units', 'timepoint'],
      dtype='object')

In [13]:
merged_metadata.to_excel("/dev/shm/mouse-development.xlsx", index=False)

In [23]:
tissues = set(merged_metadata["Biosample term name"])
tissues

{'adrenal gland',
 'embryonic facial prominence',
 'forebrain',
 'heart',
 'hindbrain',
 'intestine',
 'kidney',
 'limb',
 'liver',
 'lung',
 'midbrain',
 'neural tube',
 'skeletal muscle tissue',
 'spleen',
 'stomach',
 'thymus',
 'urinary bladder'}

In [15]:
set(merged_metadata["timepoint"])

{'embryonic 10.5 day',
 'embryonic 11.5 day',
 'embryonic 12.5 day',
 'embryonic 13.5 day',
 'embryonic 14.5 day',
 'embryonic 15.5 day',
 'embryonic 16.5 day',
 'postnatal 0 day'}

In [21]:
gene_quantifications = merged_metadata[merged_metadata["File output type"] == "gene quantifications"]
reps = gene_quantifications.groupby(["Biosample term name", "timepoint"])["File accession"].count()

In [17]:
def str_or_blank(value):
    if pandas.isnull(value):
        return ""
    else:
        return "{}".format(value)

cm = {}
for i, row in reps.reset_index().iterrows():
    cm.setdefault(row["Biosample term name"], {})[row["timepoint"]] = str_or_blank(row["File accession"])
pandas.DataFrame(cm).fillna("")

,adrenal gland,embryonic facial prominence,forebrain,heart,hindbrain,intestine,kidney,limb,liver,lung,midbrain,neural tube,skeletal muscle tissue,spleen,stomach,thymus,urinary bladder
postnatal 0 day,2,,2,2,2,2,2,,2,2,2,2,2,2,2,2,2
embryonic 10.5 day,,2,2,2,2,,,2,,,2,,,,,,
embryonic 11.5 day,,2,2,2,2,,,2,2,,2,2,,,,,
embryonic 12.5 day,,2,2,2,2,,,2,2,,2,2,,,,,
embryonic 13.5 day,,2,2,2,2,,,2,2,,2,2,,,,,
embryonic 14.5 day,,2,2,2,2,2,2,2,2,2,2,2,,,2,,
embryonic 15.5 day,,2,2,2,2,2,2,2,2,2,2,2,,,2,,
embryonic 16.5 day,,,2,2,2,2,2,,2,2,2,,,,2,,


In [18]:
time_order = [
    "embryonic 10.5 day",
    "embryonic 11.5 day",
    "embryonic 12.5 day",
    "embryonic 13.5 day",
    "embryonic 14.5 day",
    "embryonic 15.5 day",
    "embryonic 16.5 day",
    "postnatal 0 day",
]

In [19]:
reps

Biosample term name          timepoint         
adrenal gland                postnatal 0 day       2
embryonic facial prominence  embryonic 10.5 day    2
                             embryonic 11.5 day    2
                             embryonic 12.5 day    2
                             embryonic 13.5 day    2
                                                  ..
stomach                      embryonic 15.5 day    2
                             embryonic 16.5 day    2
                             postnatal 0 day       2
thymus                       postnatal 0 day       2
urinary bladder              postnatal 0 day       2
Name: File accession, Length: 78, dtype: int64

In [22]:
gene_quantifications

,File accession,File dataset,File type,File format,File output type,Assay term name,Biosample term id,Biosample term name,Biosample type,Dataset accession,...,Assembly,File status,Derived from,S3 URL,Size,biosample_term_name,mouse_life_stage,age,age_units,timepoint
10,ENCFF302TQO,ENCSR304RDL,tsv,tsv,gene quantifications,RNA-seq,UBERON:0001890,forebrain,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF832FEC",https://encode-public.s3.amazonaws.com/2016/08...,8645668,forebrain,embryonic,10.5,day,embryonic 10.5 day
14,ENCFF413BXV,ENCSR304RDL,tsv,tsv,gene quantifications,RNA-seq,UBERON:0001890,forebrain,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF919TMT",https://encode-public.s3.amazonaws.com/2016/08...,8644450,forebrain,embryonic,10.5,day,embryonic 10.5 day
28,ENCFF996YFF,ENCSR764OPZ,tsv,tsv,gene quantifications,RNA-seq,UBERON:0001891,midbrain,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF027MVS",https://encode-public.s3.amazonaws.com/2016/08...,8648419,midbrain,embryonic,10.5,day,embryonic 10.5 day
30,ENCFF421TRR,ENCSR764OPZ,tsv,tsv,gene quantifications,RNA-seq,UBERON:0001891,midbrain,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF707ILK",https://encode-public.s3.amazonaws.com/2016/08...,8652250,midbrain,embryonic,10.5,day,embryonic 10.5 day
42,ENCFF070GIC,ENCSR943LKA,tsv,tsv,gene quantifications,RNA-seq,UBERON:0002028,hindbrain,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF386IQR",https://encode-public.s3.amazonaws.com/2016/08...,8641522,hindbrain,embryonic,10.5,day,embryonic 10.5 day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,ENCFF795XBQ,ENCSR331XCE,tsv,tsv,gene quantifications,RNA-seq,UBERON:0000160,intestine,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF870XNG",https://encode-public.s3.amazonaws.com/2016/08...,8635287,intestine,postnatal,0,day,postnatal 0 day
1179,ENCFF905HUL,ENCSR173PJN,tsv,tsv,gene quantifications,RNA-seq,UBERON:0002113,kidney,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF193HAL",https://encode-public.s3.amazonaws.com/2016/08...,8647433,kidney,postnatal,0,day,postnatal 0 day
1181,ENCFF783LVC,ENCSR173PJN,tsv,tsv,gene quantifications,RNA-seq,UBERON:0002113,kidney,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF991WXK",https://encode-public.s3.amazonaws.com/2016/08...,8652801,kidney,postnatal,0,day,postnatal 0 day
1195,ENCFF465YOS,ENCSR739PEB,tsv,tsv,gene quantifications,RNA-seq,UBERON:0002369,adrenal gland,tissue,ENCSR574CRQ,...,mm10,released,"ENCFF717NFD, ENCFF077IZN",https://encode-public.s3.amazonaws.com/2016/08...,8644586,adrenal gland,postnatal,0,day,postnatal 0 day


In [28]:
for biosample_term in tissues:
    for timepoint in time_order:
        current_biosample = (gene_quantifications["Biosample term name"] == biosample_term)
        current_timepoint = (gene_quantifications["timepoint"] == timepoint)
        for i, row in gene_quantifications[current_biosample & current_timepoint].iterrows():
            print(biosample_term, timepoint, row["S3 URL"])

spleen postnatal 0 day https://encode-public.s3.amazonaws.com/2016/08/10/d5baea54-b538-445c-bec6-7a6721274d6f/ENCFF600ZQX.tsv
spleen postnatal 0 day https://encode-public.s3.amazonaws.com/2016/08/10/54b6f7d7-4aef-47d8-9109-1622afe0dc9c/ENCFF871WCS.tsv
adrenal gland postnatal 0 day https://encode-public.s3.amazonaws.com/2016/08/10/3da1298a-7c61-4d3e-9e5e-cbcef0bf05a9/ENCFF465YOS.tsv
adrenal gland postnatal 0 day https://encode-public.s3.amazonaws.com/2016/08/10/4346502c-de1e-4d3b-8aa3-dbebf36f6252/ENCFF516EUX.tsv
heart embryonic 10.5 day https://encode-public.s3.amazonaws.com/2016/08/04/18edb46a-4898-4ab6-bd9f-7654c877e3c4/ENCFF127FPD.tsv
heart embryonic 10.5 day https://encode-public.s3.amazonaws.com/2016/08/04/97347873-d26f-427b-b0d9-315f7bd7c2d4/ENCFF721XZC.tsv
heart embryonic 11.5 day https://encode-public.s3.amazonaws.com/2016/08/10/e1b3c7a2-445a-4ba5-b559-afeb7ebdabb7/ENCFF226IWR.tsv
heart embryonic 11.5 day https://encode-public.s3.amazonaws.com/2016/08/10/57cea523-7e2d-4b4b-ae52

In [27]:
row.index

Index(['File accession', 'File dataset', 'File type', 'File format',
       'File output type', 'Assay term name', 'Biosample term id',
       'Biosample term name', 'Biosample type', 'Dataset accession',
       'Dataset date released', 'Project', 'Lab', 'md5sum', 'dbxrefs',
       'File download URL', 'Assembly', 'File status', 'Derived from',
       'S3 URL', 'Size', 'biosample_term_name', 'mouse_life_stage', 'age',
       'age_units', 'timepoint'],
      dtype='object')

In [29]:
df = pandas.read_csv("https://encode-public.s3.amazonaws.com/2016/08/04/97347873-d26f-427b-b0d9-315f7bd7c2d4/ENCFF721XZC.tsv", sep="\t", index_col=None)
df

,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,FPKM_ci_lower_bound,FPKM_ci_upper_bound
0,10000,10000,72.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000
1,10001,10001,73.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000
2,10002,10002,73.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000
3,10003,10003,75.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000
4,10004,10004,78.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69686,gSpikein_ERCC-00165,tSpikein_ERCC-00165,872.0,773.0,740.0,34.90,23.31,740.0,0.0,34.22,23.28,3.168140e+01,36.664300,2.155640e+01,24.948900
69687,gSpikein_ERCC-00168,tSpikein_ERCC-00168,1024.0,925.0,9.0,0.35,0.24,9.0,0.0,0.39,0.26,1.626110e-01,0.621491,1.106970e-01,0.423000
69688,gSpikein_ERCC-00170,tSpikein_ERCC-00170,1023.0,924.0,299.0,11.80,7.88,299.0,0.0,11.59,7.89,1.027330e+01,12.889100,6.994650e+00,8.775670
69689,gSpikein_ERCC-00171,tSpikein_ERCC-00171,505.0,406.0,27528.0,2471.97,1650.87,27528.0,0.0,2420.65,1646.86,2.391850e+03,2449.020000,1.627610e+03,1666.440000
